# IIIFマニフェストファイルからTEIのfacsimile要素を作成するプログラム

利用条件： https://creativecommons.org/publicdomain/zero/1.0/

## 関数のロード

canvasのIDの末尾をxml:idとして扱います。データや用途に応じて、適宜変更してください。

In [1]:
from bs4 import BeautifulSoup
import json
import requests
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 関数

def createXml(manifest, id_format="{}"):

  baseText = r'''
  <facsimile></facsimile>
  '''

  soup = BeautifulSoup(baseText, "xml")

  facsimile = soup.find("facsimile")
  facsimile["source"] = manifest

  # manifestのロード

  manifest_data = requests.get(manifest).json()
  canvases = manifest_data["sequences"][0]["canvases"]

  for canvas in canvases:
      prefix = canvas["images"][0]["resource"]["service"]["@id"]
      image = prefix + "/full/full/0/default.jpg"
      canvasId = canvas["@id"]

      # canvasのIDの末尾をxml:idとして扱います。データや用途に応じて、適宜変更してください。
      id = id_format.format(canvasId.split("/")[-1])

      surface_text = r'''
      <surface source="{}" xml:id="{}">
      <graphic url="{}"/>
      </surface>
      '''.format(canvasId, id, image)

      surface = BeautifulSoup(surface_text, "xml")
      facsimile.append(surface)

  return soup

def main(manifest, output_file, id_format="{}"):
  # 以下、実行
  xml = createXml(manifest, id_format)

  f = open(output_file, "w")
  f.write(xml.prettify())
  f.close()

  # ファイルのダウンロード
  from google.colab import files
  files.download(output_file)

## 実行

マニフェストファイルのURLと出力ファイルのパスを指定してください。IIIF Image APIに対応していることを前提とします。

また、xml:idの与え方のフォーマットをid_format引数で指定できます。

In [ ]:
# 延喜式（国立歴史民俗博物館）の例
manifest = "https://khirin-a.rekihaku.ac.jp/iiif/rekihaku/H-743-74-1/manifest.json"
output_file = "engishiki.xml"
main(manifest, output_file)

In [2]:
# 延喜式（国立歴史民俗博物館）の例
manifest = "https://khirin-a.rekihaku.ac.jp/iiif/rekihaku/H-743-74-29/manifest.json"
output_file = "engishiki_v29_facsimile.xml"
main(manifest, output_file)

ModuleNotFoundError: No module named 'google'

In [ ]:
# 源氏物語（国立国会図書館）の例
manifest = "https://www.dl.ndl.go.jp/api/iiif/2585098/manifest.json"
output_file = "genji.xml"
main(manifest, output_file, id_format="page{}")